In [1]:
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
import random as python_random

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, BatchNormalization, Activation
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras import backend as K

### y label 0,1

In [2]:
y = pd.read_csv("new_label.csv")
print(len(y.loc[y["new_label"] == 0]))
print(len(y.loc[y["new_label"] == 1]))
y = y["new_label"]
#y.head()
len(y[y==0])

39438
2757


39438

### symptom（，385）

In [3]:
disA_fea_mat = pd.read_csv("new_disA_fea_mat.csv", index_col=0, header="infer")
disB_fea_mat = pd.read_csv("new_disB_fea_mat.csv", index_col=0, header="infer")

# disA_fea_mat = pd.read_csv("concat_mesh_label/con_feaA.csv",index_col=0,header="infer")
# disB_fea_mat = pd.read_csv("concat_mesh_label/con_feaB.csv",index_col=0,header="infer")
disB_fea_mat[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
actinomycetales infections,1.93364,5.559312,72.041217,60.463165,0.000000,13.653691,3.965454,22.610653,0.000000,0.000000,...,0.000000,19.321535,45.152099,8.997955,11.683219,18.331389,0.0,0.0,2.1545,14.206270
adenocarcinoma,5.80092,8.338968,26.247575,36.153233,2.165972,6.826846,0.000000,53.018082,8.108319,1.881661,...,2.433349,11.592921,84.283917,33.742333,14.604023,2.156634,0.0,0.0,2.1545,7.103135


### model

In [4]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes           # shape1是batch_size ?
    return (shape1[0], 1)

class euclLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(euclLayer,self).__init__(name='final_out')
        
    def __call__(self, x1, x2):
        def euclidean_distance(x1,x2):
            sum_square = K.sum(K.square(x1-x2), axis=1, keepdims=True)
            return  K.sqrt(K.maximum(sum_square, K.epsilon()))
        
        value = euclidean_distance(x1,x2)
        return value

def e_auc(y_true, y_pred):
    def euclidean_AUC(y_true,y_pred):
        #tf.compat.v1.disable_eager_execution()
        #with tf.compat.v1.Session() as sess:
            #fpr,tpr,threshold = roc_curve(y_true.eval(),y_pred.eval())
        max_pre = np.max(y_pred)
        N_y_pred = [(max_pre-i)/max_pre for i in y_pred]
        
        roc_auc = roc_auc_score(y_true,N_y_pred)
        return roc_auc

    return tf.py_function(euclidean_AUC, (y_true, y_pred),Tout=float)

class eucl_AUC(tf.keras.metrics.Metric): 
    def __init__(self, name='euclidean_AUC', **kwargs):# self.属性是tf.Variable
        super(eucl_AUC, self).__init__(name=name, **kwargs)
        self.AUC = self.add_weight(name='auc', initializer=tf.zeros_initializer())
        self.yt = self.add_weight(name='y_true', initializer=tf.zeros_initializer())
        self.yp = self.add_weight(name='y_pred', initializer=tf.zeros_initializer())
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        
        y_true = tf.concat([self.yt,y_true],axis=0)
        y_pred = tf.concat([self.yp,y_pred],axis=0)
        
        N_y_pred = tf.add(tf.multiply(tf.tanh(y_pred),-1.0),1.0)
        
        #y_true = tf.reshape(y_true,[-1])
        #N_y_pred = tf.reshape(N_y_pred,[-1])
        
        roc_auc = tf.compat.v1.metrics.auc(self.yt,N_y_pred)
        #roc_auc = tf.py_function(func= roc_auc_score,inp=[y_true,N_y_pred],Tout=[tf.float32])
        
        #roc_auc = tf.convert_to_tensor(roc_auc)
        #roc_auc = tf.cast(roc_auc,tf.float32)
        #roc_auc = tf.reshape(roc_auc,[1,1])
        
        self.yt.assign(y_true)
        self.yp.assign(y_pred)
        self.AUC.assign(roc_auc)
        
    def result(self):
        return self.AUC

####################################################################################################################
def create_base_network_copy(input_shape):
    '''Base network to be shared (eq. to feature extraction).'''
    input = Input(shape=input_shape)
    
    x= Activation(activation='relu')(Dense(128)(input))
#     x= Activation(activation='relu')(Dense(64)(input))
    x = Dropout(0.1)(x)
    
    x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    
    x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    
    x= Activation(activation='relu')(Dense(32)(x))
    x = Dropout(0.1)(x)
    
#     x= Activation(activation='relu')(Dense(32)(x))
#     x = Dropout(0.1)(x)
    
    x= Activation(activation='relu')(Dense(32)(x))
    x = Dropout(0.1)(x)

    x= Activation(activation='sigmoid')(Dense(32)(x))
#     x = Dense(32)(x)
    
    return Model(input, x,name="base_net")

def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).'''
    input = Input(shape=input_shape)
    
    x= Activation(activation='relu')(Dense(128)(input))
#     x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    
    x= Activation(activation='relu')(Dense(64)(x))
    x = Dropout(0.1)(x)
    
    x= Activation(activation='relu')(Dense(32)(x))
    x = Dropout(0.1)(x)

#     x= Activation(activation='sigmoid')(Dense(32)(x))
    x= Activation(activation='sigmoid')(Dense(32)(x))
    return Model(input, x,name="base_net")

def create_euclLayer(input_shape):
    pair1 = Input(shape=(input_shape))
    pair2 = Input(shape=(input_shape))
              
    x = euclLayer()(pair1,pair2)
    x = Activation(activation='relu')(x)
#     x = Activation(activation='sigmoid')(x)
    
    return Model(inputs=[pair1,pair2],outputs=x,name="siamese")
    
"""state-of-the-art：input(64)-64-32-16-1"""
def create_MLP_copy(input_shape):
    
    input = Input(shape=input_shape)
    x= Activation(activation='relu')(Dense(64)(input))
#     x= Activation(activation='relu')(Dense(32)(input))
    x= Activation(activation='relu')(Dense(32)(x))
    x= Activation(activation='relu')(Dense(16)(x))
#     x= Activation(activation='relu')(Dense(8)(x))
    
#     x= Activation(activation='sigmoid',name="MLP_out")(Dense(1)(x))
    x= Dense(1,activation='sigmoid')(x)
    
    return Model(input, x ,name="mlp")

def create_MLP(input_shape):
    
    input = Input(shape=input_shape)
    x= Activation(activation='relu')(Dense(64)(input))
    
    x= Activation(activation='relu')(Dense(32)(x))
    
#     x= Activation(activation='relu')(Dense(32)(x))
    x= Activation(activation='relu')(Dense(16)(x))
    
    x= Activation(activation='relu')(Dense(8)(x))
    
#     x= Activation(activation='sigmoid',name="MLP_out")(Dense(1)(x))
    x= Dense(1,activation='sigmoid')(x)
    
    return Model(input, x ,name="mlp")

## SDSP

In [5]:
mesh_label_A = pd.read_csv("concat_mesh_label/mesh_label_A_new.csv",index_col=0)
mesh_label_B = pd.read_csv("concat_mesh_label/mesh_label_B_new.csv",index_col=0)
print(mesh_label_B.shape)
mesh_label_B[:2]

(42195, 24)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
actinomycetales infections,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adenocarcinoma,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:

mesh_label_A=mesh_label_A.astype(np.float32)
mesh_label_B=mesh_label_B.astype(np.float32)

y=y.astype(np.float32)


y=np.array(y)

disA_fea_mat=np.array(disA_fea_mat)
disB_fea_mat=np.array(disB_fea_mat)

mesh_label_A = np.array(mesh_label_A)
mesh_label_B = np.array(mesh_label_B)

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
i =1


neg_to_pos= 1
rus = RandomUnderSampler(sampling_strategy=neg_to_pos,random_state=1) 
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y2 = rus.fit_resample(X=disB_fea_mat,y=y)
print('rus_y的长度:',len(rus_y))

rus_mesh_A,rus_y3= rus.fit_resample(X=mesh_label_A,y=y)
rus_mesh_B,rus_y4 = rus.fit_resample(X=mesh_label_B,y=y)
print("rus_mesh_B的长度：",len(rus_mesh_B))

seed = 1
np.random.seed(seed) 
python_random.seed(seed)
tf.random.set_seed(seed)


Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=rus_dfmA,y=rus_y):
    ###############################################################################################
 
    input_shape=(385)
    base_network = create_base_network(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    input_mesh_a = Input(shape=24)
    input_mesh_b = Input(shape=24)
    

    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    

    eucl_model = create_euclLayer(processed_a.shape[-1])
    eucl_out = eucl_model([processed_a,processed_b]) 
    
    
    concat_fea = tf.keras.layers.concatenate([processed_a,input_mesh_a,processed_b,input_mesh_b],axis=-1)
    print("concat_fea的shape：",concat_fea.shape) 
    
    MLP = create_MLP(concat_fea.shape[-1])
    MLP_out = MLP(concat_fea)
    
    model = Model(inputs=[input_a,input_b,input_mesh_a,input_mesh_b], outputs=[eucl_out,MLP_out])
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
        
    #####################################################################################################################

    print("-------------------------------------Kfold: {} iter-----------------------------------------".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
    my_loss_weight = {"siamese":1,"mlp":0.1}
    my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)
    
    model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
        loss= my_loss,
        loss_weights=my_loss_weight,
        metrics=my_metrics,
        
        optimizer=rms,
    )
    
    # fit()中 shuffle=True
    model.fit([rus_dfmA[train],rus_dfmB[train],rus_mesh_A[train],rus_mesh_B[train]],
              [rus_y[train],rus_y[train]],
              
              batch_size=None,
              epochs=200,
              callbacks=callback,
              shuffle=True,
             )
    
    scores = model.evaluate([rus_dfmA[test],rus_dfmB[test],rus_mesh_A[test],rus_mesh_B[test]],
                            [rus_y[test],rus_y[test]],
                            verbose=1,
                            batch_size=32)
    
    predict_res = model.predict([rus_dfmA[test],rus_dfmB[test],rus_mesh_A[test],rus_mesh_B[test]])
    
    cvacc.append(scores[-2] * 100)
    cvauc_mlp.append(scores[-1] * 100)
    cvauc_sia.append(scores[-3] * 100)
    
    print("------ %s: %.2f%%\t ----- %s: %.2f%%" % 
           (model.metrics_names[-2],scores[-2]*100, model.metrics_names[-1],scores[-1]*100))
     
print("十折性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc_mlp), np.std(cvauc_mlp)))


rus_y的长度: 5514
rus_mesh_B的长度： 5514


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


concat_fea的shape： (None, 112)
-------------------------------------Kfold: 1 iter-----------------------------------------
Epoch 1/200
156/156 [==============================] - 0s 2ms/step - loss: 1.3188 - siamese_loss: 1.2517 - mlp_loss: 0.6711 - mlp_binary_accuracy: 0.5941 - mlp_auc_10: 0.6253
Epoch 2/200
156/156 [==============================] - 0s 2ms/step - loss: 0.6960 - siamese_loss: 0.6327 - mlp_loss: 0.6328 - mlp_binary_accuracy: 0.6538 - mlp_auc_10: 0.6977
Epoch 3/200
156/156 [==============================] - 0s 2ms/step - loss: 0.6488 - siamese_loss: 0.5879 - mlp_loss: 0.6083 - mlp_binary_accuracy: 0.6634 - mlp_auc_10: 0.7301
Epoch 4/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5977 - siamese_loss: 0.5395 - mlp_loss: 0.5821 - mlp_binary_accuracy: 0.6943 - mlp_auc_10: 0.7617
Epoch 5/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5963 - siamese_loss: 0.5401 - mlp_loss: 0.5621 - mlp_binary_accuracy: 0.7146 - mlp_auc_10: 0.7834
Epo

156/156 [==============================] - 0s 2ms/step - loss: 0.5503 - siamese_loss: 0.4883 - mlp_loss: 0.6199 - mlp_binary_accuracy: 0.6653 - mlp_auc_12: 0.7143
Epoch 4/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5464 - siamese_loss: 0.4861 - mlp_loss: 0.6030 - mlp_binary_accuracy: 0.6765 - mlp_auc_12: 0.7387
Epoch 5/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5368 - siamese_loss: 0.4782 - mlp_loss: 0.5863 - mlp_binary_accuracy: 0.6951 - mlp_auc_12: 0.7583
Epoch 6/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5026 - siamese_loss: 0.4464 - mlp_loss: 0.5626 - mlp_binary_accuracy: 0.7211 - mlp_auc_12: 0.7819
Epoch 7/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5310 - siamese_loss: 0.4765 - mlp_loss: 0.5449 - mlp_binary_accuracy: 0.7356 - mlp_auc_12: 0.7994
Epoch 8/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5312 - siamese_loss: 0.4776 - mlp_loss: 0.5360 - mlp_binary_

156/156 [==============================] - 0s 2ms/step - loss: 0.5406 - siamese_loss: 0.4979 - mlp_loss: 0.4267 - mlp_binary_accuracy: 0.8044 - mlp_auc_15: 0.8859
Epoch 27/200
18/18 [==============================] - 0s 2ms/step - loss: 0.4605 - siamese_loss: 0.4095 - mlp_loss: 0.5101 - mlp_binary_accuracy: 0.7768 - mlp_auc_15: 0.8455
------ mlp_binary_accuracy: 77.68%	 ----- mlp_auc_15: 84.55%
concat_fea的shape： (None, 112)
-------------------------------------Kfold: 7 iter-----------------------------------------
Epoch 1/200
156/156 [==============================] - 0s 2ms/step - loss: 1.4940 - siamese_loss: 1.4256 - mlp_loss: 0.6843 - mlp_binary_accuracy: 0.5527 - mlp_auc_16: 0.5819
Epoch 2/200
156/156 [==============================] - 0s 2ms/step - loss: 0.6808 - siamese_loss: 0.6165 - mlp_loss: 0.6433 - mlp_binary_accuracy: 0.6333 - mlp_auc_16: 0.6816
Epoch 3/200
156/156 [==============================] - 0s 2ms/step - loss: 0.5862 - siamese_loss: 0.5243 - mlp_loss: 0.6195 - mlp_

156/156 [==============================] - 0s 2ms/step - loss: 0.4639 - siamese_loss: 0.4161 - mlp_loss: 0.4777 - mlp_binary_accuracy: 0.7687 - mlp_auc_19: 0.8526
Epoch 20/200
156/156 [==============================] - 0s 2ms/step - loss: 0.4771 - siamese_loss: 0.4299 - mlp_loss: 0.4716 - mlp_binary_accuracy: 0.7769 - mlp_auc_19: 0.8569
Epoch 21/200
156/156 [==============================] - 0s 2ms/step - loss: 0.4489 - siamese_loss: 0.4027 - mlp_loss: 0.4616 - mlp_binary_accuracy: 0.7830 - mlp_auc_19: 0.8631
Epoch 22/200
156/156 [==============================] - 0s 2ms/step - loss: 0.4328 - siamese_loss: 0.3868 - mlp_loss: 0.4602 - mlp_binary_accuracy: 0.7902 - mlp_auc_19: 0.8640
Epoch 23/200
156/156 [==============================] - 0s 2ms/step - loss: 0.4209 - siamese_loss: 0.3748 - mlp_loss: 0.4607 - mlp_binary_accuracy: 0.7872 - mlp_auc_19: 0.8647
Epoch 24/200
156/156 [==============================] - 0s 2ms/step - loss: 0.4644 - siamese_loss: 0.4182 - mlp_loss: 0.4619 - mlp_bi

In [65]:
print(len(mesh_label_A[train])/5514)
print(len(test))
print(test[-100:-1])
print(len(mesh_label_A))
print(len(rus_mesh_A))
print(rus_y3[:100])
print(rus_y4[-100:-1])

0.9000725426187886
551
[4544 4550 4555 4593 4606 4620 4651 4654 4666 4677 4681 4686 4699 4711
 4714 4725 4743 4748 4756 4782 4811 4814 4828 4833 4844 4851 4881 4886
 4887 4896 4914 4927 4928 4939 4962 4964 4968 4970 4981 4982 4985 5005
 5020 5023 5026 5037 5040 5047 5060 5081 5084 5085 5087 5107 5115 5116
 5129 5152 5169 5170 5178 5202 5204 5225 5233 5235 5241 5262 5264 5267
 5288 5289 5294 5296 5299 5314 5324 5326 5332 5336 5337 5346 5372 5377
 5384 5387 5394 5400 5404 5412 5425 5435 5444 5450 5456 5461 5476 5481
 5496]
42195
5514
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 

In [119]:
"""test virtual mesh feature"""
rand_matA = nvp.random.randint(1,size=(42195,24))
rand_matB = np.random.randint(1,size=(42195,24))

for i in range(42195):
    rand_matA[i][np.random.randint(24)]=1
    rand_matB[i][np.random.randint(24)]=1
print(rand_matA[:1])
print(rand_matB[:1])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]


###  results
#### test virtual mesh feature
    1、undersampling-------- ave_acc: 79.47% (+/- 3.32%)	 ----- ave_auc_mlp:87.95% (+/- 2.23%)
    2、random 0/1
        all random-------- ave_acc: 66.03% (+/- 2.48%)	 ----- ave_auc_mlp:72.06% (+/- 1.61%)
        one random 1 for each disease mesh feature-------- ave_acc: 70.82% (+/- 3.30%)	 ----- ave_auc_mlp:77.01% (+/- 3.62%)
        shuffle mesh one-hot-------- ave_acc: 71.38% (+/- 1.90%)	 ----- ave_auc_mlp:77.56% (+/- 2.78%)
#### test loss weights
    sia:1,mlp:1:-------- ave_acc: 74.14% (+/- 4.36%)	 ----- ave_auc_mlp:83.77% (+/- 3.00%)
    sia:0.1,mlp:1:-------- ave_acc: 76.91% (+/- 2.57%)	 ----- ave_auc_mlp:85.23% (+/- 1.73%)
    sia:0.01,mlp:1:-------- ave_acc: 77.98% (+/- 2.79%)	 ----- ave_auc_mlp:86.88% (+/- 2.80%)
    
    sia:1,mlp:0.1:-------- ave_acc: 73.87% (+/- 2.61%)	 ----- ave_auc_mlp:83.78% (+/- 1.85%)

## ramdom split test dataset

In [16]:

mesh_label_A=mesh_label_A.astype(np.float32)
mesh_label_B=mesh_label_B.astype(np.float32)

y=y.astype(np.float32)

y=np.array(y)

disA_fea_mat=np.array(disA_fea_mat)
disB_fea_mat=np.array(disB_fea_mat)

mesh_label_A = np.array(mesh_label_A)
mesh_label_B = np.array(mesh_label_B)

In [15]:
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
cvacc_test,cvauc_mlp_test,cvauc_sia_test=[],[],[]
i =1


neg_to_pos= 1
rus = RandomUnderSampler(sampling_strategy=neg_to_pos,random_state=2) 
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)

rus_mesh_A,rus_y = rus.fit_resample(X=mesh_label_A,y=y)
rus_mesh_B,rus_y = rus.fit_resample(X=mesh_label_B,y=y)

print('rus_y的长度:',len(rus_y))

"""split 20%"""
symA_train,symA_test,symB_train,symB_test,meshA_train,meshA_test,meshB_train,meshB_test,y_train,y_test=train_test_split(
                rus_dfmA,rus_dfmB,
                rus_mesh_A,rus_mesh_B,#mesh_label_A[-5515:-1],mesh_label_B[-5515:-1],
                rus_y,
                test_size=0.2, shuffle=True)


seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)


Kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
for train,test in Kfold.split(X=symA_train,y=y_train):
    ###############################################################################################

    input_shape=(385)
    base_network = create_base_network(input_shape)
    
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    input_mesh_a = Input(shape=24)
    input_mesh_b = Input(shape=24)
    

    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    

    eucl_model = create_euclLayer(processed_a.shape[-1])
    eucl_out = eucl_model([processed_a,processed_b]) 
    

    concat_fea = tf.keras.layers.concatenate([processed_a,input_mesh_a,processed_b,input_mesh_b],axis=-1)
    print("concat_fea的shape：",concat_fea.shape) #(None,32)
    MLP = create_MLP(concat_fea.shape[-1])
    MLP_out = MLP(concat_fea)
    
    model = Model(inputs=[input_a, input_b,input_mesh_a,input_mesh_b], outputs=[eucl_out,MLP_out])
    
    """L2"""
    for layer in model.layers:
        if hasattr(layer,'kernel_regularizer'):
            layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
        
    #####################################################################################################################

    print("-------------------------------------Kfold: {} iter-----------------------------------------".format(i))
    i+=1

    rms = RMSprop()
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)
    
    my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
    my_loss_weight = {"siamese":0,"mlp":1}
    my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)
    
    model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
        loss= my_loss,
        loss_weights=my_loss_weight,
        metrics=my_metrics,
        
        optimizer=rms,
    )
    
    # fit()中 shuffle=True
    model.fit([symA_train[train],symB_train[train],meshA_train[train],meshB_train[train]],
              [y_train[train],y_train[train]],
              
              batch_size=None,
              epochs=200,  #200-->50-->100
              callbacks=callback,
              shuffle=True,
             )
    
#symA_train,symA_test,symB_train,symB_test,meshA_train,meshA_test,meshB_train,meshB_test,y_train,y_test    
    valid_scores = model.evaluate([symA_train[test],symB_train[test],meshA_train[test],meshB_train[test]],
                            [y_train[test],y_train[test]],
                            verbose=1,
                            batch_size=32)
    
    cvacc.append(valid_scores[-2] * 100)
    cvauc_mlp.append(valid_scores[-1] * 100)
    cvauc_sia.append(valid_scores[-3] * 100)
    
    test_scores = model.evaluate([symA_test,symB_test,meshA_test,meshB_test],
                            [y_test,y_test],
                            verbose=1,
                            batch_size=32)
    
    cvacc_test.append(test_scores[-2] * 100)
    cvauc_mlp_test.append(test_scores[-1] * 100)
    cvauc_sia_test.append(test_scores[-3] * 100)
    
    print("验证集十折------ %s: %.2f%%\t ----- %s: %.2f%% \n" % 
           (model.metrics_names[-2],valid_scores[-2]*100, model.metrics_names[-1],valid_scores[-1]*100))
    print("测试集------ %s: %.2f%%\t ----- %s: %.2f%% \n" % 
           (model.metrics_names[-2],test_scores[-2]*100, model.metrics_names[-1],test_scores[-1]*100))
     
print("训练集十折性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc), np.std(cvacc),np.mean(cvauc_mlp), np.std(cvauc_mlp)))
print("测试集性能均值：-------- ave_acc: %.2f%% (+/- %.2f%%)\t ----- ave_auc_mlp:%.2f%% (+/- %.2f%%)" % 
      (np.mean(cvacc_test), np.std(cvacc_test),np.mean(cvauc_mlp_test), np.std(cvauc_mlp_test)))

rus_y的长度: 5514


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


concat_fea的shape： (None, 112)
-------------------------------------Kfold: 1 iter-----------------------------------------
Epoch 1/200
125/125 [==============================] - 0s 3ms/step - loss: 0.6741 - siamese_loss: 2.7143 - mlp_loss: 0.6741 - mlp_binary_accuracy: 0.5734 - mlp_auc_56: 0.6131
Epoch 2/200
125/125 [==============================] - 0s 2ms/step - loss: 0.6408 - siamese_loss: 1.9223 - mlp_loss: 0.6408 - mlp_binary_accuracy: 0.6344 - mlp_auc_56: 0.6852
Epoch 3/200
125/125 [==============================] - 0s 2ms/step - loss: 0.6275 - siamese_loss: 1.5244 - mlp_loss: 0.6275 - mlp_binary_accuracy: 0.6513 - mlp_auc_56: 0.7042
Epoch 4/200
125/125 [==============================] - 0s 2ms/step - loss: 0.6075 - siamese_loss: 1.9501 - mlp_loss: 0.6075 - mlp_binary_accuracy: 0.6803 - mlp_auc_56: 0.7298
Epoch 5/200
125/125 [==============================] - 0s 2ms/step - loss: 0.5939 - siamese_loss: 1.9327 - mlp_loss: 0.5939 - mlp_binary_accuracy: 0.6888 - mlp_auc_56: 0.7473
Epo

125/125 [==============================] - 0s 2ms/step - loss: 0.5725 - siamese_loss: 3.3354 - mlp_loss: 0.5725 - mlp_binary_accuracy: 0.7161 - mlp_auc_57: 0.7723
Epoch 7/200
125/125 [==============================] - 0s 2ms/step - loss: 0.5636 - siamese_loss: 3.1829 - mlp_loss: 0.5636 - mlp_binary_accuracy: 0.7237 - mlp_auc_57: 0.7807
Epoch 8/200
125/125 [==============================] - 0s 2ms/step - loss: 0.5534 - siamese_loss: 3.2229 - mlp_loss: 0.5534 - mlp_binary_accuracy: 0.7227 - mlp_auc_57: 0.7917
Epoch 9/200
125/125 [==============================] - 0s 2ms/step - loss: 0.5457 - siamese_loss: 3.3737 - mlp_loss: 0.5457 - mlp_binary_accuracy: 0.7257 - mlp_auc_57: 0.7993
Epoch 10/200
125/125 [==============================] - 0s 2ms/step - loss: 0.5442 - siamese_loss: 2.8013 - mlp_loss: 0.5442 - mlp_binary_accuracy: 0.7320 - mlp_auc_57: 0.7998
Epoch 11/200
125/125 [==============================] - 0s 3ms/step - loss: 0.5260 - siamese_loss: 3.0159 - mlp_loss: 0.5260 - mlp_binar

125/125 [==============================] - 0s 2ms/step - loss: 0.4346 - siamese_loss: 2.9336 - mlp_loss: 0.4346 - mlp_binary_accuracy: 0.8023 - mlp_auc_58: 0.8787
Epoch 25/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4158 - siamese_loss: 3.4386 - mlp_loss: 0.4158 - mlp_binary_accuracy: 0.8086 - mlp_auc_58: 0.8897
Epoch 26/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4234 - siamese_loss: 3.2218 - mlp_loss: 0.4234 - mlp_binary_accuracy: 0.8098 - mlp_auc_58: 0.8854
Epoch 27/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4079 - siamese_loss: 3.3683 - mlp_loss: 0.4079 - mlp_binary_accuracy: 0.8149 - mlp_auc_58: 0.8946
Epoch 28/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4053 - siamese_loss: 3.6281 - mlp_loss: 0.4053 - mlp_binary_accuracy: 0.8227 - mlp_auc_58: 0.8949
Epoch 29/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4048 - siamese_loss: 3.7414 - mlp_loss: 0.4048 - mlp_bi

125/125 [==============================] - 0s 2ms/step - loss: 0.3446 - siamese_loss: 4.3452 - mlp_loss: 0.3446 - mlp_binary_accuracy: 0.8481 - mlp_auc_59: 0.9259
Epoch 53/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3577 - siamese_loss: 4.4026 - mlp_loss: 0.3577 - mlp_binary_accuracy: 0.8436 - mlp_auc_59: 0.9198
Epoch 54/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3390 - siamese_loss: 4.0133 - mlp_loss: 0.3390 - mlp_binary_accuracy: 0.8481 - mlp_auc_59: 0.9284
Epoch 55/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3402 - siamese_loss: 4.2713 - mlp_loss: 0.3402 - mlp_binary_accuracy: 0.8486 - mlp_auc_59: 0.9277
Epoch 56/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3389 - siamese_loss: 4.1572 - mlp_loss: 0.3389 - mlp_binary_accuracy: 0.8511 - mlp_auc_59: 0.9283
Epoch 57/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3257 - siamese_loss: 4.2745 - mlp_loss: 0.3257 - mlp_bi

125/125 [==============================] - 0s 2ms/step - loss: 0.4123 - siamese_loss: 3.8506 - mlp_loss: 0.4123 - mlp_binary_accuracy: 0.8144 - mlp_auc_61: 0.8921
Epoch 33/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4061 - siamese_loss: 3.5107 - mlp_loss: 0.4061 - mlp_binary_accuracy: 0.8118 - mlp_auc_61: 0.8952
Epoch 34/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4048 - siamese_loss: 3.2898 - mlp_loss: 0.4048 - mlp_binary_accuracy: 0.8156 - mlp_auc_61: 0.8963
Epoch 35/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4004 - siamese_loss: 3.7146 - mlp_loss: 0.4004 - mlp_binary_accuracy: 0.8156 - mlp_auc_61: 0.8981
Epoch 36/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3946 - siamese_loss: 3.4758 - mlp_loss: 0.3946 - mlp_binary_accuracy: 0.8139 - mlp_auc_61: 0.9020
Epoch 37/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3948 - siamese_loss: 3.6611 - mlp_loss: 0.3948 - mlp_bi

125/125 [==============================] - 0s 2ms/step - loss: 0.5052 - siamese_loss: 3.2885 - mlp_loss: 0.5052 - mlp_binary_accuracy: 0.7501 - mlp_auc_63: 0.8319
Epoch 15/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4961 - siamese_loss: 3.2304 - mlp_loss: 0.4961 - mlp_binary_accuracy: 0.7627 - mlp_auc_63: 0.8392
Epoch 16/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4942 - siamese_loss: 2.9109 - mlp_loss: 0.4942 - mlp_binary_accuracy: 0.7637 - mlp_auc_63: 0.8409
Epoch 17/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4801 - siamese_loss: 2.9937 - mlp_loss: 0.4801 - mlp_binary_accuracy: 0.7730 - mlp_auc_63: 0.8508
Epoch 18/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4782 - siamese_loss: 2.9118 - mlp_loss: 0.4782 - mlp_binary_accuracy: 0.7688 - mlp_auc_63: 0.8514
Epoch 19/200
125/125 [==============================] - 0s 2ms/step - loss: 0.4720 - siamese_loss: 3.2966 - mlp_loss: 0.4720 - mlp_bi

125/125 [==============================] - 0s 2ms/step - loss: 0.3876 - siamese_loss: 3.8956 - mlp_loss: 0.3876 - mlp_binary_accuracy: 0.8277 - mlp_auc_64: 0.9055
Epoch 43/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3815 - siamese_loss: 4.1472 - mlp_loss: 0.3815 - mlp_binary_accuracy: 0.8292 - mlp_auc_64: 0.9086
Epoch 44/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3767 - siamese_loss: 4.0478 - mlp_loss: 0.3767 - mlp_binary_accuracy: 0.8330 - mlp_auc_64: 0.9111
Epoch 45/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3678 - siamese_loss: 4.0360 - mlp_loss: 0.3678 - mlp_binary_accuracy: 0.8401 - mlp_auc_64: 0.9152
Epoch 46/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3719 - siamese_loss: 4.3154 - mlp_loss: 0.3719 - mlp_binary_accuracy: 0.8395 - mlp_auc_64: 0.9135
Epoch 47/200
125/125 [==============================] - 0s 2ms/step - loss: 0.3832 - siamese_loss: 4.3750 - mlp_loss: 0.3832 - mlp_bi

####   resluts
    1、10-fold avg：-------- ave_acc: 80.30% (+/- 2.14%)	 ----- ave_auc_mlp:87.53% (+/- 2.44%)
    test avg：-------- ave_acc: 78.97% (+/- 1.52%)	 ----- ave_auc_mlp:87.12% (+/- 0.96%)
    2、10-fold avg：-------- ave_acc: 79.46% (+/- 3.21%)	 ----- ave_auc_mlp:87.38% (+/- 2.56%)
    test avg：-------- ave_acc: 78.36% (+/- 1.83%)	 ----- ave_auc_mlp:86.72% (+/- 1.13%)

In [52]:
# symA_train,symA_test,symB_train,symB_test,meshA_train,meshA_test,meshB_train,meshB_test
print(len(symA_train))
print(len(symA_test))
print(len(symB_train))
print(len(symB_test))
print(len(meshA_train))
print(len(meshB_test))

4411
1103
4411
1103
4411
1103


### using all data train model

In [18]:
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics  import roc_curve,auc,roc_auc_score
from imblearn.under_sampling import RandomUnderSampler

cvacc,cvauc_mlp,cvpre,cvpre_top,cvrec,cvauc_sia=[],[],[],[],[],[]
cvacc_test,cvauc_mlp_test,cvauc_sia_test=[],[],[]
i =1


neg_to_pos= 1
rus = RandomUnderSampler(sampling_strategy=neg_to_pos,random_state=2) 
rus_dfmA,rus_y = rus.fit_resample(X=disA_fea_mat,y=y)
rus_dfmB,rus_y = rus.fit_resample(X=disB_fea_mat,y=y)

rus_mesh_A,rus_y = rus.fit_resample(X=mesh_label_A,y=y)
rus_mesh_B,rus_y = rus.fit_resample(X=mesh_label_B,y=y)

print('rus_y的长度:',len(rus_y))


seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.
tf.random.set_seed(seed)


###############################################################################################

input_shape=(385)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
input_mesh_a = Input(shape=24)
input_mesh_b = Input(shape=24)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

eucl_model = create_euclLayer(processed_a.shape[-1])
eucl_out = eucl_model([processed_a,processed_b]) 


concat_fea = tf.keras.layers.concatenate([processed_a,input_mesh_a,processed_b,input_mesh_b],axis=-1)
print("concat_fea的shape：",concat_fea.shape) #(None,32)
MLP = create_MLP(concat_fea.shape[-1])
MLP_out = MLP(concat_fea)

model = Model(inputs=[input_a, input_b,input_mesh_a,input_mesh_b], outputs=[eucl_out,MLP_out])


for layer in model.layers:
    if hasattr(layer,'kernel_regularizer'):
        layer.kernel_regularizer = tf.keras.regularizers.l2(0.001)
    
#####################################################################################################################

rms = RMSprop()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)

my_loss = {"siamese":contrastive_loss,"mlp":tf.keras.losses.BinaryCrossentropy()}
my_loss_weight = {"siamese":0,"mlp":1}
my_metrics ={ "mlp":[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()]} #tf.keras.metrics.Precision(top_k=len(rus_y)//100)

model.compile(
#         loss= [tfa.losses.ContrastiveLoss(),tf.keras.losses.BinaryCrossentropy()],
#         loss_weights=[0.01,1],
#         metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()],
        
    loss= my_loss,
    loss_weights=my_loss_weight,
    metrics=my_metrics,
    
    optimizer=rms,
)

# fit()中 shuffle=True
model.fit([rus_dfmA,rus_dfmB,rus_mesh_A,rus_mesh_B],
          [rus_y,rus_y],
          
          batch_size=None,
          epochs=200,  #200-->50-->100
          callbacks=callback,
          shuffle=True,
         )



rus_y的长度: 5514


'PYTHONHASHSEED' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


concat_fea的shape： (None, 112)
Epoch 1/200
173/173 [==============================] - 0s 2ms/step - loss: 0.6731 - siamese_loss: 3.7768 - mlp_loss: 0.6731 - mlp_binary_accuracy: 0.5805 - mlp_auc_66: 0.6140
Epoch 2/200
173/173 [==============================] - 0s 2ms/step - loss: 0.6352 - siamese_loss: 3.1242 - mlp_loss: 0.6352 - mlp_binary_accuracy: 0.6467 - mlp_auc_66: 0.6891
Epoch 3/200
173/173 [==============================] - 0s 2ms/step - loss: 0.6117 - siamese_loss: 2.9967 - mlp_loss: 0.6117 - mlp_binary_accuracy: 0.6656 - mlp_auc_66: 0.7221
Epoch 4/200
173/173 [==============================] - 0s 2ms/step - loss: 0.5923 - siamese_loss: 2.5803 - mlp_loss: 0.5923 - mlp_binary_accuracy: 0.6846 - mlp_auc_66: 0.7486
Epoch 5/200
173/173 [==============================] - 0s 2ms/step - loss: 0.5842 - siamese_loss: 2.8422 - mlp_loss: 0.5842 - mlp_binary_accuracy: 0.6922 - mlp_auc_66: 0.7583
Epoch 6/200
173/173 [==============================] - 0s 3ms/step - loss: 0.5690 - siamese_los

In [19]:
# model.save("Undersampling_model.h5")

### predict 42195 disease pairs

In [21]:
predict_result = model.predict([disA_fea_mat,disB_fea_mat,mesh_label_A,mesh_label_B])
print(len(predict_result))

2


In [28]:
print(len(predict_result[0]))
print(len(predict_result[1]))
print(predict_result[0])
print(predict_result[1])

42195
42195
[[5.       ]
 [4.898979 ]
 [3.8729832]
 ...
 [1.7320508]
 [4.8989797]
 [5.       ]]
[[2.6254924e-05]
 [1.3631827e-01]
 [4.7320038e-02]
 ...
 [1.1805803e-02]
 [5.6417070e-07]
 [1.9087295e-07]]


In [31]:
# pd.DataFrame(predict_result[1],columns=["predict_score"]).to_csv("model_predict42195_result.csv")

### predict disease pair similarities of 4268 diseases

In [51]:
dis_sym_fea = pd.read_csv("../../python爬虫/2020_TF-IDF_dropdup.csv",index_col=0)
dis_sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
dis_mesh_fea = pd.read_csv("Mesh_files/label_array_new.csv",index_col=0)
dis_mesh_fea[:2]

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
"mesothelioma, cystic",0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
palatal neoplasms,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
from tqdm import tqdm

symA=[]
symB=[]
meshA=[]
meshB=[]

for disA in tqdm(dis_sym_fea.index):
    for disB in dis_sym_fea.index:
        symA.append(dis_sym_fea.loc[disA])
        symB.append(dis_sym_fea.loc[disB])
        
        meshA.append(dis_mesh_fea.loc[disA.lower()])
        meshB.append(dis_mesh_fea.loc[disB.lower()])
        
pd.DataFrame(symA).to_csv("all_symA.csv")
pd.DataFrame(symB).to_csv("all_symB.csv")
pd.DataFrame(meshA).to_csv("all_meshA.csv")
pd.DataFrame(meshB).to_csv("all_meshB.csv")

  2%|█▌                                                                            | 86/4268 [02:46<2:14:49,  1.93s/it]


KeyboardInterrupt: 